In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ***Analisis_Sentimiento:***

### Importar librerias

In [29]:
import pandas as pd
import pyarrow as py
from textblob import TextBlob
import re
import nltk
from nltk.corpus import stopwords
from textblob import Word
import pyarrow.parquet as pq

In [14]:
# Descargar recursos de NLTK
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True



---

### Cargar el DataFrame df_user_review

In [16]:
# Se genera una variable con el archivo parquet
df_user_reviews = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/PI_1/Data/df_user_reviews.parquet')


In [20]:
# Visualizacion del DataFrame
df_user_reviews

,Id_Usuario,Url_Usuario,funny,Posteo,Ultima_Edicion,Id_Item,Util,Recomendado,Review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,posted november 5 2011,,1250,no ratings yet,true,simple yet with great replayability in my opin...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,posted july 15 2011,,22200,no ratings yet,true,its unique and worth a playthrough
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,posted april 21 2011,,43110,no ratings yet,true,great atmosphere the gunplay can be a bit chun...
3,js41637,http://steamcommunity.com/id/js41637,,posted june 24 2014,,251610,15 of 20 people (75%) found this review helpful,true,i know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,posted september 8 2013,,227300,0 of 1 people (0%) found this review helpful,true,for a simple (its actually not all that simple...
...,...,...,...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,posted july 10,,70,no ratings yet,true,a must have classic from steam definitely wort...
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,posted july 8,,362890,no ratings yet,true,this game is a perfect remake of the original ...
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,posted july 3,,273110,1 of 2 people (50%) found this review helpful,true,had so much fun plaing this and collecting res...
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,posted july 20,,730,no ratings yet,true,:d



### Crea funcion para limpiar el texto


In [4]:
# Función para limpiar el texto
def clean_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar caracteres especiales y puntuación
    text = ' '.join([word for word in text.split() if word not in (stopwords.words('english'))])  # Eliminar palabras de parada
    text = ' '.join([Word(word).lemmatize() for word in text.split()])  # Lematización

    return text



---



---


### Crea funcion para Analisis de Sentimiento


In [21]:
# Definir una función para el análisis de sentimiento
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity < 0:
        return 0  # Malo
    elif analysis.sentiment.polarity == 0:
        return 1  # Neutral
    else:
        return 2  # Positivo

In [22]:
# Aplicar limpieza a la columna 'Review'
df_user_reviews['clean_reviews'] = df_user_reviews['Review'].apply(clean_text)

In [23]:
# Aplicar el análisis de sentimiento a la columna 'clean_reviews' y crear una nueva columna 'analisis_sentimientos'
df_user_reviews['analisis_sentimientos'] = df_user_reviews['clean_reviews'].apply(get_sentiment)

In [27]:
# Visualizacion del dataFrame
df_user_reviews.head(100)

,Id_Usuario,Url_Usuario,funny,Posteo,Ultima_Edicion,Id_Item,Util,Recomendado,clean_reviews,analisis_sentimientos
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,posted november 5 2011,,1250,no ratings yet,true,simple yet great replayability opinion zombie ...,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,posted july 15 2011,,22200,no ratings yet,true,unique worth playthrough,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,posted april 21 2011,,43110,no ratings yet,true,great atmosphere gunplay bit chunky time end d...,2
3,js41637,http://steamcommunity.com/id/js41637,,posted june 24 2014,,251610,15 of 20 people (75%) found this review helpful,true,know think see title barbie dreamhouse party i...,2
4,js41637,http://steamcommunity.com/id/js41637,,posted september 8 2013,,227300,0 of 1 people (0%) found this review helpful,true,simple actually simple truck driving simulator...,0
5,js41637,http://steamcommunity.com/id/js41637,,posted november 29 2013,,239030,1 of 4 people (25%) found this review helpful,true,fun little game play bored time passer gud rec...,0
6,evcentric,http://steamcommunity.com/id/evcentric,,posted february 3,,248820,no ratings yet,true,suitably punishing roguelike platformer winnin...,2
7,evcentric,http://steamcommunity.com/id/evcentric,,posted december 4 2015,last edited december 5 2015,370360,no ratings yet,true,run fun hell kind fun,2
8,evcentric,http://steamcommunity.com/id/evcentric,,posted november 3 2014,,237930,no ratings yet,true,elegant integration gameplay story world devel...,2
9,evcentric,http://steamcommunity.com/id/evcentric,,posted october 15 2014,,263360,no ratings yet,true,random drop random quest stat point animation ...,0


In [25]:
# Se elimina la columna 'Review'
df_user_reviews = df_user_reviews.drop(['Review'], axis=1)

### Guardar el DataFrame comprimido en Parquet


In [31]:
# Guardar el DataFrame comprimido en Parquet
df_user_reviews.to_parquet('df_user_reviews_sentimientos.parquet')